In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import sklearn

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
# 1. DataFrame.info()를 통해 결측값이 있는지 확인(Null
# 2. layer Node의 수는 대부분 512가 적당하다. (1024 가끔)
# 3. cost가 진동(증가,감소 반복)하면 lr를 줄인다.

In [4]:
# 랜덤 시드 설정
torch.manual_seed(1)
if device == "cuda" :
    torch.cuda.manual_seed_all(1)

In [6]:
# csv 데이터 로드
train = pd.read_csv("2021-ai-midterm-p2/train.csv")
test = pd.read_csv("2021-ai-midterm-p2/test.csv")
submission = pd.read_csv("2021-ai-midterm-p2/submit_sample.csv")

# 데이터 확인
print(train)
print(train.info())

      ID      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0      0   0.14150   0.0   6.91     0  0.448  6.169   6.6  5.7209    3  233   
1      1   0.15445  25.0   5.13     0  0.453  6.145  29.2  7.8148    8  284   
2      2  16.81180   0.0  18.10     0  0.700  5.277  98.1  1.4261   24  666   
3      3   0.05646   0.0  12.83     0  0.437  6.232  53.7  5.0141    5  398   
4      4   8.79212   0.0  18.10     0  0.584  5.565  70.6  2.0635   24  666   
..   ...       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
399  399   0.03548  80.0   3.64     0  0.392  5.876  19.1  9.2203    1  315   
400  400   0.09164   0.0  10.81     0  0.413  6.065   7.8  5.2873    4  305   
401  401   5.87205   0.0  18.10     0  0.693  6.405  96.0  1.6768   24  666   
402  402   0.33045   0.0   6.20     0  0.507  6.086  61.5  3.6519    8  307   
403  403   0.08014   0.0   5.96     0  0.499  5.850  41.5  3.9342    5  279   

     PTRATIO       B  LSTST  MEDV  
0       17.9  3

In [8]:
# 사용하지 않은 컬럼을 제거하여 Train, Test 데이터 설정
x_train = np.array(train.drop(["ID","MEDV"],axis=1))
x_test = np.array(test.drop(["ID"],axis=1))
y_train = np.array(train["MEDV"])

In [9]:
# 데이터 전처리
# 값의 범위가 크므로 정규화가 필요
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [11]:
# 데이터 모양 확인
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

torch.Size([404, 13])
torch.Size([404])
torch.Size([102, 13])


In [10]:
# 데이터 Tensor에 올리기
x_train = torch.Tensor(x_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_train = torch.Tensor(y_train).to(device)

In [15]:
# 레이어 생성
layer1 = torch.nn.Linear(13,512,bias=True).to(device)
layer2 = torch.nn.Linear(512,512,bias=True).to(device)
layer3 = torch.nn.Linear(512,512,bias=True).to(device)
layer4 = torch.nn.Linear(512,1,bias=True).to(device)
relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.3).to(device)

In [16]:
# 레이어 가중치 초기화
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)

Parameter containing:
tensor([[ 1.5340e-02, -1.9601e-01, -2.3885e-03,  6.9688e-02, -4.6459e-02,
         -1.5758e-02,  6.8991e-02,  6.3995e-02,  5.7575e-02,  6.0135e-02,
          3.1797e-02,  5.6753e-02, -1.0249e-01, -2.1797e-02, -2.5634e-02,
          1.7749e-02,  1.2800e-01, -2.9713e-02,  1.7944e-02, -2.2922e-02,
         -1.0625e-01,  7.4894e-02, -3.5503e-02,  1.3233e-01,  5.7287e-02,
          2.7309e-03, -7.0738e-02, -3.2921e-02, -1.6033e-02, -1.9586e-03,
         -1.1982e-01, -4.5819e-02, -1.7347e-02,  1.1795e-02, -2.9652e-02,
          3.3305e-02,  5.5894e-02, -2.2570e-03,  1.8555e-01,  3.0921e-02,
          4.2561e-02, -1.2277e-02,  7.9325e-02,  7.7262e-02,  4.6076e-02,
         -5.1701e-02,  5.6731e-02,  1.4382e-02,  5.9282e-02,  2.0580e-02,
         -1.6031e-02,  7.2544e-02,  1.6943e-02, -3.4517e-02,  2.5863e-02,
         -7.8040e-02,  4.2454e-02, -4.4802e-02, -1.6548e-01,  5.9937e-02,
         -7.6594e-02, -1.1570e-01,  4.0454e-04, -1.6126e-03, -4.9168e-02,
         -6.3926

In [17]:
# 모델 생성
model = torch.nn.Sequential(layer1, relu, dropout,
                           layer2, relu, dropout,
                           layer3, relu, dropout,
                           layer4).to(device)

In [19]:
# 학습 파라미터 설정
lr = 0.001
epochs = 5000

In [20]:
# 손실 함수, 최적화 함수 정의
loss = torch.nn.MSELoss().to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)

In [21]:
# 학습
model.train()
for epoch in range(epochs+1):
    optim.zero_grad()
    h = model(x_train)
    cost = loss(h,y_train.unsqueeze(1))
    cost.backward()
    optim.step()
    if epoch % (epochs/10) == 0:
        print(epoch, cost.item())

0 592.19580078125
500 7.759638786315918
1000 4.611252307891846
1500 3.5548384189605713
2000 3.7512996196746826
2500 2.720580816268921
3000 3.794879198074341
3500 3.397120237350464
4000 2.560767412185669
4500 3.640705108642578
5000 3.352344512939453


In [23]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
    predict = predict.cpu().detach().numpy()
submission["MEDV"] = predict
print(submission)

      ID       MEDV      label
0      0  30.148865  30.148865
1      1  25.292566  25.292566
2      2  17.454540  17.454540
3      3  20.880470  20.880470
4      4  22.421289  22.421289
..   ...        ...        ...
97    97  25.885521  25.885521
98    98  41.879757  41.879757
99    99  22.924595  22.924595
100  100  12.347336  12.347336
101  101  16.669802  16.669802

[102 rows x 3 columns]


In [45]:
submission.to_csv("submission.csv",index=False)